In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
checkpoints = '/content/drive/MyDrive/colab_files/birds/'
if not os.path.exists(checkpoints):
    os.makedirs(checkpoints)

# Data Download

In [ ]:
import os
if not os.path.exists('birds21wi'):
    !mkdir birds21wi
    os.chdir('birds21wi')
    !wget https://pjreddie.com/media/files/birds/train.tar
    !wget https://pjreddie.com/media/files/birds/test.tar
    !wget https://pjreddie.com/media/files/birds/names.txt
    !tar xf train.tar
    !tar xf test.tar
    !mkdir testing
    !mv test testing
    os.chdir('..')

In [ ]:
def get_bird_data(augmentation=0):
    val_ratio = 0.1  # make validation set 10% of training set size

    # original size: 128
    transform_train = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomCrop(224, padding=8, padding_mode='edge'), # Take 128x128 crops from padded images
        transforms.RandomHorizontalFlip(),    # 50% of time flip image along y-axis
        transforms.ToTensor(),
    ])

    transform_val = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomCrop(224, padding=8, padding_mode='edge'), # Take 128x128 crops from padded images
        transforms.RandomHorizontalFlip(),    # 50% of time flip image along y-axis
        transforms.ToTensor(),
    ])
    
    transform_test = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
    ])

    # load training dataset
    trainset = torchvision.datasets.ImageFolder(root='birds21wi/train', transform=transform_train)

    # create class organizers
    classes = open("birds21wi/names.txt").read().strip().split("\n")
    class_to_idx = trainset.class_to_idx
    idx_to_class = {int(v): int(k) for k, v in class_to_idx.items()}
    idx_to_name = {k: classes[v] for k,v in idx_to_class.items()}

    # split training set into training and validation set
    valset_len = int(len(trainset) * val_ratio)
    trainset_len = len(trainset) - valset_len
    valset, trainset = torch.utils.data.random_split(trainset, [valset_len, trainset_len])

    # create train and val loaders
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
    valloader = torch.utils.data.DataLoader(valset, batch_size=32, shuffle=True, num_workers=2)

    # load test dataset
    testset = torchvision.datasets.ImageFolder(root='birds21wi/testing', transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)

    print("Trainset length: ", trainset_len)
    print("Valset length: ", valset_len)
    print("Testset length: ", len(testset))

    
    return {'train': trainloader, 'test': testloader, 'val': valloader, 'to_class': idx_to_class, 'to_name':idx_to_name}

data = get_bird_data()

In [ ]:
print(data['to_class'])
print(data['to_name'])

In [ ]:
dataiter = iter(data['train'])
images, labels = dataiter.next()
images = images[:8]
print(images.size())

val_iter = iter(data['val'])
val_images, val_labels = val_iter.next()
val_images = val_images[:8]
print(val_images.size())

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# show images
print("Training set")
imshow(torchvision.utils.make_grid(images))
# print labels
print("Labels:" + ', '.join('%9s' % data['to_name'][labels[j].item()] for j in range(8)))

# show images
print("Validation set")
imshow(torchvision.utils.make_grid(val_images))
# print labels
print("Labels:" + ', '.join('%9s' % data['to_name'][val_labels[j].item()] for j in range(8)))

# Training + Predicting

In [ ]:
def train(net, dataloader, val_dataloader, epochs=1, start_epoch=0, lr=0.01, momentum=0.9, decay=0.0005, 
          verbose=1, print_every=10, state=None, schedule={}, checkpoint_path=None):
    net.to(device)
    # net.train()
    losses = []
    val_losses = []
    valset_len = 3856
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=decay)

    # Load previous training state
    if state:
        net.load_state_dict(state['net'])
        optimizer.load_state_dict(state['optimizer'])
        start_epoch = state['epoch']
        losses = state['losses']

    # Fast forward lr schedule through already trained epochs
    for epoch in range(start_epoch):
        if epoch in schedule:
            print ("Learning rate: %f"% schedule[epoch])
            for g in optimizer.param_groups:
                g['lr'] = schedule[epoch]

    for epoch in range(start_epoch, epochs):
        sum_loss = 0.0

        print("Training for epoch: ", epoch)
        # Update learning rate when scheduled
        if epoch in schedule:
            print ("Learning rate: %f"% schedule[epoch])
            for g in optimizer.param_groups:
                g['lr'] = schedule[epoch]

        # training data
        for i, batch in enumerate(dataloader, 0):
            inputs, labels = batch[0].to(device), batch[1].to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # autograd magic, computes all the partial derivatives
            optimizer.step() # takes a step in gradient direction

            losses.append(loss.item())
            sum_loss += loss.item()

            if i % print_every == print_every-1:
                if verbose:
                  print('[%d, %5d] train loss: %.3f' % (epoch, i + 1, sum_loss / print_every))
                sum_loss = 0.0
        
        print()
        # ------------------------------------------------------------------------
        val_sum_loss = 0.0
        val_running_correct = 0.0

        print("Validation for epoch: ", epoch)
        # validation data
        for i, batch in enumerate(val_dataloader, 0):
          inputs, labels = batch[0].to(device), batch[1].to(device)

          optimizer.zero_grad()

          with torch.set_grad_enabled(False):
            outputs = net(inputs)
            _, preds = torch.max(outputs.data, 1)
            val_loss = criterion(outputs, labels)

          val_losses.append(val_loss.item())
          val_sum_loss += val_loss.item()
          val_running_correct += torch.sum(preds == labels.data)

          if i % print_every == print_every-1:
              if verbose:
                print('[%d, %5d] val loss: %.3f' % (epoch, i + 1, val_sum_loss / print_every))
              val_sum_loss = 0.0
       
        # print validation accuracy per epoch
        val_epoch_acc = val_running_correct.double() / valset_len
        print("Val Accuracy: {:.4f}".format(val_epoch_acc))
        print()

        if checkpoint_path:
            state = {'epoch': epoch+1, 'net': net.state_dict(), 'optimizer': optimizer.state_dict(), 'losses': losses}
            torch.save(state, checkpoint_path + 'checkpoint-%d.pkl'%(epoch+1))

    return losses

In [ ]:
def predict(net, dataloader, ofname):
    out = open(ofname, 'w')
    out.write("path,class\n")
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader, 0):
            if i%100 == 0:
                print(i)
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            fname, _ = dataloader.dataset.samples[i]
            out.write("test/{},{}\n".format(fname.split('/')[-1], data['to_class'][predicted.item()]))
    out.close()

# ResNet18 Experiment

## ResNet18 Training

In [ ]:
resnet = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
resnet.fc = nn.Linear(512, 555) # This will reinitialize the layer as well

losses = train(resnet, data['train'], data['val'], epochs=5, lr=.001, decay=.001, print_every=50, checkpoint_path=checkpoints)

In [ ]:
resnet = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
resnet.fc = nn.Linear(512, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-5.pkl')
losses = train(resnet, data['train'], data['val'], epochs=10, schedule={0:.01, 8:.001}, lr=.01, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

In [ ]:
def smooth(x, size):
  return np.convolve(x, np.ones(size)/size, mode='valid')
plt.plot(smooth(losses,50))

In [ ]:
state = torch.load(checkpoints + 'checkpoint-6.pkl')
plt.plot(smooth(state['losses'], 50))

## ResNet18 Test

In [ ]:
# Load model from checkpoint
resnet = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
resnet.fc = nn.Linear(512, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-6.pkl')
resnet.load_state_dict(state['net'])


predict(resnet, data['test'], checkpoints + "preds.csv")

# ResNet34 Experiment

## ResNet34 Train

In [ ]:
resnet34 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=True)
resnet34.fc = nn.Linear(512, 555) # This will reinitialize the layer as well

losses = train(resnet34, data['train'], data['val'], epochs=5, lr=.001, decay=0.001, print_every=50, checkpoint_path=checkpoints)

In [ ]:
resnet34 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=True)
resnet34.fc = nn.Linear(512, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-5.pkl')
losses = train(resnet34, data['train'], data['val'], epochs=10, lr=.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

## ResNet34 Test

In [ ]:
# Load model from checkpoint
resnet34 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet34', pretrained=True)
resnet34.fc = nn.Linear(512, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-10.pkl')
resnet34.load_state_dict(state['net'])


predict(resnet34, data['test'], checkpoints + "preds.csv")

# ResNet50 Experiment

## ResNet50 Train

In [ ]:
resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well

losses = train(resnet50, data['train'], data['val'], epochs=5, lr=.001, print_every=50, checkpoint_path=checkpoints)

In [ ]:
resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-5.pkl')
losses = train(resnet50, data['train'], data['val'], epochs=10, lr=.001, print_every=50, checkpoint_path=checkpoints, state=state)

In [ ]:
resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-10.pkl')
losses = train(resnet50, data['train'], data['val'], epochs=20, schedule={0:.001, 4:.0001}, lr=.001, decay=0.005, print_every=50, checkpoint_path=checkpoints, state=state)

## ResNet50 Test

In [ ]:
# Load model from checkpoint
resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-6.pkl')
resnet50.load_state_dict(state['net'])


predict(resnet50, data['test'], checkpoints + "preds.csv")

# ResNet50 Experiment with Hyperparameters

## ResNet50 Training - Hyperparams

In [ ]:
resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well

losses = train(resnet50, data['train'], data['val'], epochs=5, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints)

In [ ]:
resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-5.pkl')
losses = train(resnet50, data['train'], data['val'], epochs=10, schedule={0:.001, 4:.0001, 8: .00001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

In [ ]:
resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-10.pkl')
losses = train(resnet50, data['train'], data['val'], epochs=20, schedule={0:.001, 4:.0001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

## ResNet50 Test - Hyperparams

In [ ]:
# Load model from checkpoint
resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-20.pkl')
resnet50.load_state_dict(state['net'])


predict(resnet50, data['test'], checkpoints + "preds.csv")zx

# ResNet101 Experiment

## ResNet101 Train

In [ ]:
resnet101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet101', pretrained=True)
resnet101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well

losses = train(resnet101, data['train'], data['val'], epochs=5, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints)

In [ ]:
resnet101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet101', pretrained=True)
resnet101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-5.pkl')
losses = train(resnet101, data['train'], data['val'], epochs=10, schedule={0:.01, 4:0.001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

In [ ]:
resnet101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet101', pretrained=True)
resnet101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-10.pkl')
losses = train(resnet101, data['train'], data['val'], epochs=20, schedule={0:.01, 4:0.001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

## ResNet101 Test

In [ ]:
# Load model from checkpoint
resnet101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet101', pretrained=True)
resnet101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-20.pkl')
resnet101.load_state_dict(state['net'])


predict(resnet101, data['test'], checkpoints + "preds.csv")zx

# ResNet152 Experiment

## ResNet152 Training

In [ ]:
resnet152 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
resnet152.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well

losses = train(resnet152, data['train'], data['val'], epochs=5, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints)

In [ ]:
resnet152 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
resnet152.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-5.pkl')
losses = train(resnet152, data['train'], data['val'], epochs=10, schedule={0:.01, 6:.0001, 12:.00001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

In [ ]:
resnet152 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
resnet152.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-18.pkl')
losses = train(resnet152, data['train'], data['val'], epochs=25, schedule={0:.01, 4:.001}, lr=0.0001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

## ResNet152 Test

In [ ]:
# Load model from checkpoint
resnet152 = torch.hub.load('pytorch/vision:v0.9.0', 'resnet152', pretrained=True)
resnet152.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-25.pkl')
resnet152.load_state_dict(state['net'])

predict(resnet152, data['test'], checkpoints + "preds.csv")
print("Finished creating .csv file!")

# Wide ResNet50 Experiment

## Wide ResNet50 Train

In [ ]:
wide_resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=True)
wide_resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well

losses = train(wide_resnet50, data['train'], data['val'], epochs=5, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints)

In [ ]:
wide_resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=True)
wide_resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-5.pkl')
losses = train(wide_resnet50, data['train'], data['val'], epochs=10, schedule={0:.01, 4:.001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

In [ ]:
wide_resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=True)
wide_resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-10.pkl')
losses = train(wide_resnet50, data['train'], data['val'], epochs=20, schedule={0:.01, 4:.001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

## Wide ResNet50 Test

In [ ]:
# Load model from checkpoint
wide_resnet50 = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=True)
wide_resnet50.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-20.pkl')
wide_resnet50.load_state_dict(state['net'])

predict(wide_resnet50, data['test'], checkpoints + "preds.csv")
print("Finished creating .csv file!")

# ResNext101 Experiment

## ResNext101 Training

In [ ]:
resnext101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
resnext101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well

losses = train(resnext101, data['train'], data['val'], epochs=5, lr=0.01, decay=0.001, print_every=50, checkpoint_path=checkpoints)

In [ ]:
resnext101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
resnext101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-5.pkl')
losses = train(resnext101, data['train'], data['val'], epochs=10, schedule={0:.01, 4:.001, 12:.0001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

In [ ]:
resnext101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
resnext101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-10.pkl')
losses = train(resnext101, data['train'], data['val'], epochs=15, schedule={0:.01, 4:.001, 8:.0001}, lr=0.001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

In [ ]:
resnext101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
resnext101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-15.pkl')
losses = train(resnext101, data['train'], data['val'], epochs=25, schedule={0:.01, 4:.001, 8:.0001}, lr=0.0001, decay=0.001, print_every=50, checkpoint_path=checkpoints, state=state)

## ResNext101 Test

In [ ]:
# Load model from checkpoint
resnext101 = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
resnext101.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-28.pkl')
resnext101.load_state_dict(state['net'])

predict(resnext101, data['test'], checkpoints + "preds.csv")
print("Finished creating .csv file!")